In [1]:
from typing import List

from punctuators.models import SBDModelONNX

In [2]:
with open("content/habr.txt", "r", encoding="utf-8") as f:
    txt = f.read()

In [6]:
m = SBDModelONNX.from_pretrained("sbd_multi_lang")

input_texts: List[str] = [
    txt
]

results: List[List[str]] = m.infer(input_texts)

Input: P-tune — если кратко, это автоматизированный подбор промпта (более детально об этом рассказал коллега в этой статье). Он работает эффективнее, чем человек, которого посадили подбирать удачный промпт. В целом это хороший подход, но в нашем случае он почти никогда не давал требуемых результатов с необходимым качеством. Сам подход не может именно научить модель: задача должна быть достаточно простой для модели или в каком-то виде присутствовать в обучении.

Для сложных задач интереснее LoRA и fine-tune. На практике разница между ними в максимальном качестве, которого можно достичь: у разморозки слоёв в fine-tune предел качества выше. Но давайте рассмотрим каждый подход подробнее.

У LoRA есть параметр — ранг. Чем больше ранг, тем более сложному навыку можно обучить модель. Но и тут есть свои нюансы. Мы проводили эксперимент с рангом LoRA и ростом качества в зависимости от него. Шаг ранга — степень двойки (4, 8, 16, 32, 64, 128, 256, 512). Спешу расстроить: экспоненциального роста к

# Sentence preprocess

In [3]:
import re

sentences = re.split(r'\.\s|\.\n', txt)

for i, sentence in enumerate(sentences):
    sentences[i] = sentence.strip()

In [4]:
sentences

['P-tune — если кратко, это автоматизированный подбор промпта (более детально об этом рассказал коллега в этой статье)',
 'Он работает эффективнее, чем человек, которого посадили подбирать удачный промпт',
 'В целом это хороший подход, но в нашем случае он почти никогда не давал требуемых результатов с необходимым качеством',
 'Сам подход не может именно научить модель: задача должна быть достаточно простой для модели или в каком-то виде присутствовать в обучении',
 'Для сложных задач интереснее LoRA и fine-tune',
 'На практике разница между ними в максимальном качестве, которого можно достичь: у разморозки слоёв в fine-tune предел качества выше',
 'Но давайте рассмотрим каждый подход подробнее',
 'У LoRA есть параметр — ранг',
 'Чем больше ранг, тем более сложному навыку можно обучить модель',
 'Но и тут есть свои нюансы',
 'Мы проводили эксперимент с рангом LoRA и ростом качества в зависимости от него',
 'Шаг ранга — степень двойки (4, 8, 16, 32, 64, 128, 256, 512)',
 'Спешу расстрои

# NLP

In [5]:
from transformers import BertModel, BertTokenizerFast
from datasets import Dataset
import torch
import torch.nn.functional as F
import pandas as pd
from transformers.pipelines import pipeline

from nltk import download
from nltk.corpus import stopwords

download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
russian_stopwords = set(stopwords.words("russian"))

In [7]:
from sentence_transformers import SentenceTransformer

model_checkpoint = 'paraphrase-multilingual-MiniLM-L12-v2'
hf_model = pipeline("feature-extraction", model="bert-base-multilingual-cased")

model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

embeddings = model.encode(sentences)

In [8]:
values_dict = {"index":[], "sentence": [], "embedding": []}
for i, sentence in enumerate(sentences):
    values_dict["index"].append(i)
    values_dict["sentence"].append(sentences[i])
    values_dict["embedding"].append(embeddings[i])

In [26]:
df = pd.DataFrame(values_dict)
df.head()

,index,sentence,embedding
0,0,"P-tune — если кратко, это автоматизированный п...","[-0.43873578, 0.0045500644, -0.058726426, -0.1..."
1,1,"Он работает эффективнее, чем человек, которого...","[0.015044462, -0.043138217, -0.13492848, -0.08..."
2,2,"В целом это хороший подход, но в нашем случае ...","[-0.043706495, -0.15788434, -0.10562504, -0.30..."
3,3,Сам подход не может именно научить модель: зад...,"[-0.15333599, 0.059547313, -0.103571616, -0.04..."
4,4,Для сложных задач интереснее LoRA и fine-tune,"[-0.065739356, -0.106273375, -0.18514916, 0.14..."


In [30]:
import faiss
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

dimension = len(embeddings[0])
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings).astype('float32'))

distances = []
for i in range(len(embeddings) - 1):
    distance = index.search(np.array([embeddings[i]]).astype('float32'), 2)[0][0][1]
    distances.append(distance)

df['faiss_search'] = distances + [None]

cos_dist = []
for i in range(len(embeddings) - 1):
    sim = cosine_similarity(embeddings[i].reshape(1, -1), embeddings[1+1].reshape(1, -1))[0][0]
    cos_dist.append(sim)

df['cos_sim'] = cos_dist + [None]
df['super_score'] = df['faiss_search'] * df['cos_sim']

In [31]:
df[['sentence', 'super_score']]

,sentence,super_score
0,"P-tune — если кратко, это автоматизированный п...",2.359216
1,"Он работает эффективнее, чем человек, которого...",3.913883
2,"В целом это хороший подход, но в нашем случае ...",8.857592
3,Сам подход не может именно научить модель: зад...,3.668906
4,Для сложных задач интереснее LoRA и fine-tune,2.052966
5,На практике разница между ними в максимальном ...,3.880389
6,Но давайте рассмотрим каждый подход подробнее,3.420843
7,У LoRA есть параметр — ранг,1.890748
8,"Чем больше ранг, тем более сложному навыку мож...",2.572504
9,Но и тут есть свои нюансы,2.193518


In [37]:
for i, row in enumerate(df.iterrows()):
    if i == 0:
        continue
    if df['super_score'][i] < 2.3:
        print("\nNew chapter: \n")
    print(df['sentence'][i])

Он работает эффективнее, чем человек, которого посадили подбирать удачный промпт
В целом это хороший подход, но в нашем случае он почти никогда не давал требуемых результатов с необходимым качеством
Сам подход не может именно научить модель: задача должна быть достаточно простой для модели или в каком-то виде присутствовать в обучении

New chapter: 

Для сложных задач интереснее LoRA и fine-tune
На практике разница между ними в максимальном качестве, которого можно достичь: у разморозки слоёв в fine-tune предел качества выше
Но давайте рассмотрим каждый подход подробнее

New chapter: 

У LoRA есть параметр — ранг
Чем больше ранг, тем более сложному навыку можно обучить модель

New chapter: 

Но и тут есть свои нюансы
Мы проводили эксперимент с рангом LoRA и ростом качества в зависимости от него

New chapter: 

Шаг ранга — степень двойки (4, 8, 16, 32, 64, 128, 256, 512)
Спешу расстроить: экспоненциального роста качества модели, увы, не происходит
Мы выяснили, что лучше выбирать значени

In [34]:
df['super_score']

AttributeError: 'Series' object has no attribute 'mean_25'